<a href="https://colab.research.google.com/github/TurkuNLP/DIGHT25/blob/main/01_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# DSPy tutorial part 1:

*   Grabs a dataset on climate change claims
*   Carries out the same classification task using DSPy
*   Compares the outputs

In [ ]:
!pip3 install -q 'datasets<4.0.0'
!pip3 install -q dspy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.2/41.2 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 260.1/260.1 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 90.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 400.9/400.9 kB 29.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.4/57.4 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 247.0/247.0 kB 20.0 MB/s eta 0:00:00


In [ ]:
from datasets import load_dataset
import random


#### new comment

# Load the Frugal AI Challenge dataset
dataset = load_dataset("QuotaClimat/frugalaichallenge-text-train")

# Access the training split
train_data = dataset["train"]

# Pick 10 random items
sampled_items = random.sample(list(train_data), 10)

# Print out a few samples to see the structure
for item in sampled_items[:10]: #prints 10 samples
    print(" Quote:", item["quote"])
    print(" Label:", item["label"])
    print()

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

data/train-00000-of-00001.parquet:   0%|          | 0.00/1.02M [00:00<?, ?B/s]

data/test-00000-of-00001.parquet:   0%|          | 0.00/248k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/4872 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1219 [00:00<?, ? examples/s]

 Quote: The last global warming cycle ended in 1790 and the year 2020 is 230 following this - thus I have been talking about rapid cooling beginning in 2019.
 Label: 1_not_happening

 Quote: Dismay as PM vows to ‘max out’ UK fossil fuel reserves,
 Label: 7_fossil_fuels_needed

 Quote: This book proves the aphorism that a little knowledge is dangerous. It’s nominally about air pollution. It’s really about mind pollution.
 Label: 0_not_relevant

 Quote: Vast regions of Africa's deserts at one time were covered in ocean water climate change it's been going on since the earth was formed
 Label: 2_not_human

 Quote: U.S. surface records […] show no significant warming over the past 80 years. There have been more all-time U.S. cold records than heat records since the 1940s.
 Label: 1_not_happening

 Quote: As the CO2 content of the air continues to rise, it is likely that wheat plants will exhibit significant increases in photosynthesis and yield even under less than favorable growing condit

In [ ]:
import dspy


class ClimateViewClassifier(dspy.Signature):
    """Classify a quote or claim about climate change using one of the following labels:

    Not-Relevant: No relevant claim detected or claims that don't fit other categories

    Not-Happening: Claims denying the occurrence of global warming and its effects - Global warming is not happening. Climate change is NOT leading to melting ice (such as glaciers, sea ice, and permafrost), increased extreme weather, or rising sea levels. Cold weather also shows that climate change is not happening

    Not-Human: Claims denying human responsibility in climate change - Greenhouse gases from humans are not the causing climate change.

    Not-Bad: Claims minimizing or denying negative impacts of climate change - The impacts of climate change will not be bad and might even be beneficial.

    Solutions-Harmful-Unnecessary: Claims against climate solutions - Climate solutions are harmful or unnecessary

    Science-is-Unreliable: Claims questioning climate science validity - Climate science is uncertain, unsound, unreliable, or biased.

    Proponents-Biased: Claims attacking climate scientists and activists - Climate scientists and proponents of climate action are alarmist, biased, wrong, hypocritical, corrupt, and/or politically motivated.

    Fossil-Fuels-Needed: Claims promoting fossil fuel necessity - We need fossil fuels for economic growth, prosperity, and to maintain our standard of living.
    """

    quote = dspy.InputField(desc="Quote or claim about climate change")
    label = dspy.OutputField(
        desc="Label of the quote",
        choices=["Not-Happening","Not-Human","Not-Bad","Solutions-Harmful-Unnecessary","Science-is-Unreliable","Proponents-Biased","Fossil-Fuels-Needed"],
    )

# Initialize DSPy with an LLM (use default OpenAI if configured, or HF local)
lm=dspy.LM("openai/gpt-4.1-mini",api_key="...")
dspy.configure(lm=lm)

# Instantiate classifier
classifier = dspy.Predict(signature=ClimateViewClassifier)

# Run predictions on our 30 samples
for item in sampled_items:
    quote = item["quote"]
    true_label = item["label"]
    pred = classifier(quote=quote)
    print("Text:", quote.replace("\n", " "))
    print("True:", true_label)
    print("Pred:", pred.label)
    print("-" * 40)


Text: The last global warming cycle ended in 1790 and the year 2020 is 230 following this - thus I have been talking about rapid cooling beginning in 2019.
True: 1_not_happening
Pred: Not-Happening
----------------------------------------
Text: Dismay as PM vows to ‘max out’ UK fossil fuel reserves,
True: 7_fossil_fuels_needed
Pred: Fossil-Fuels-Needed
----------------------------------------
Text: This book proves the aphorism that a little knowledge is dangerous. It’s nominally about air pollution. It’s really about mind pollution.
True: 0_not_relevant
Pred: Not-Relevant
----------------------------------------
Text: Vast regions of Africa's deserts at one time were covered in ocean water climate change it's been going on since the earth was formed
True: 2_not_human
Pred: Not-Relevant
----------------------------------------
Text: U.S. surface records […] show no significant warming over the past 80 years. There have been more all-time U.S. cold records than heat records since the 19

# Follow-up task(s)

1.   Is the model wrong, when it disagrees with the dataset annotation?
2.   Think of any other classification of these claims, and try to modify the DSPy code - did it work?
